In [36]:

from transformers import AutoModelForCausalLM, AutoTokenizer, pipelines
from accelerate import init_empty_weights
import torch

In [37]:
# Checking if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [38]:
# Create a Chain of Thought system prompt
cot_system_prompt = """You are a predicate logic assistant specializing in formal logic analysis.

Your capabilities include:
1. Evaluating the validity of logical arguments using predicate logic
2. Identifying contradictions and fallacies
3. Translating natural language into predicate logic notation
4. Assessing logical equivalence between statements
5. Performing logical operations such as negation, conjunction, disjunction, and implication

When analyzing logical problems, work through your reasoning step-by-step:
1. First, identify and formalize the premises using logical notation
2. Apply the relevant logical rules and transformations
3. Check if the conclusion follows from the premises
4. Provide your final answer as "yes" or "no"

Remember to be thorough in your analysis and explain each step of your reasoning process.
"""


In [39]:
# Create a CoT example that demonstrates the reasoning process

cot_example_1_user = "Consider the following premises: Neither Barbara is a writer nor Susan is playing a game. Can we infer the following from them? Answer yes or no: Barbara is not a writer and Susan is not playing a game."

cot_example_1_assistant ="""Let me work through this step-by-step:

1. First, I'll formalize the premise: "Neither Barbara is a writer nor Susan is playing a game."
   - Let W = "Barbara is a writer"
   - Let P = "Susan is playing a game"
   - The premise can be written as: ¬(W ∨ P), which means "not (W or P)"

2. According to De Morgan's laws: ¬(W ∨ P) ≡ (¬W ∧ ¬P)
   - This translates to: "Barbara is not a writer AND Susan is not playing a game"

3. The query asks if we can infer: "Barbara is not a writer and Susan is not playing a game"
   - This is exactly what we derived in step 2

4. Since the query statement is logically equivalent to our premise, we can infer it directly.

Therefore, the answer is: yes"""


cot_example_2_user = "Consider the following premises: The claim that Susan is running and the claim that Charles is a janitor cannot both be true. Can we infer the following from them? Answer yes or no: Jessica is writing a letter."

cot_example_2_assistant ="""Let me work through this step-by-step:

1. First, I'll formalize the premise: "The claim that Susan is running and the claim that Charles is a janitor cannot both be true."
   - Let R = "Susan is running"
   - Let J = "Charles is a janitor"
   - The premise can be written as: ¬(R ∧ J), which means "not (R and J)"
   - This is equivalent to: (¬R ∨ ¬J), meaning "Either Susan is not running OR Charles is not a janitor"

2. The query asks if we can infer: "Jessica is writing a letter."
   - Let W = "Jessica is writing a letter"

3. I need to determine if W follows logically from ¬(R ∧ J) or (¬R ∨ ¬J)
   - There is no logical connection between "Susan running or Charles being a janitor" and "Jessica writing a letter"
   - None of the established rules of inference allow us to derive W from our premises
   - These are completely unrelated statements

4. Since there's no logical connection between the premise and the query, we cannot infer the query from the premise.

Therefore, the answer is: no"""

In [40]:
import pandas as pd
dataset_path = "/home/fozle/Course Work/Y1S2/CISC 844/Projects/LLM-Predicate-Logic/dataset/data.csv"
df = pd.read_csv(dataset_path)

In [41]:
sample_size = 5000  # Adjust based on how many examples you want to test
df_sample = df.sample(sample_size, random_state=42)

In [42]:
from transformers import pipeline
import torch
pipe = pipeline(
    "text-generation",
    model="meta-llama/Llama-3.2-1B-Instruct",  # or 3B version
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

Device set to use cuda:0


In [43]:
#  load only one row from the dataset
row = df_sample.iloc[3]
row

id                                                            1737
logic                                                    predicate
rule_category                                           equivalent
rule                                existential biconditional laws
problem                                              contradiction
query            Consider the following premises: There is at l...
answer                                                          no
Name: 1736, dtype: object

In [44]:

# user_prompt = row['query']
user_prompt = row['query']
print(user_prompt)
ground_truth = row['answer']
ground_truth

Consider the following premises: There is at least one x for which if x writes a letter, then x is a lawyer and if x were a lawyer, then x is writing a letter. Can we infer the following from them? Answer yes or no: For all x, it is not true that x will write a letter if and only if x is a lawyer.


'no'

In [45]:
cot_system_prompt

'You are a predicate logic assistant specializing in formal logic analysis.\n\nYour capabilities include:\n1. Evaluating the validity of logical arguments using predicate logic\n2. Identifying contradictions and fallacies\n3. Translating natural language into predicate logic notation\n4. Assessing logical equivalence between statements\n5. Performing logical operations such as negation, conjunction, disjunction, and implication\n\nWhen analyzing logical problems, work through your reasoning step-by-step:\n1. First, identify and formalize the premises using logical notation\n2. Apply the relevant logical rules and transformations\n3. Check if the conclusion follows from the premises\n4. Provide your final answer as "yes" or "no"\n\nRemember to be thorough in your analysis and explain each step of your reasoning process.\n'

In [46]:
 # Prepare messages for the model
messages = [
        {"role": "system", "content": cot_system_prompt},
        {"role": "user", "content": cot_example_1_user},
        {"role": "assistant", "content": cot_example_1_assistant},
        {"role": "user", "content": cot_example_2_user},
        {"role": "assistant", "content": cot_example_2_assistant},
        {"role": "user", "content": user_prompt}
    ]
outputs = pipe(
            messages,
            max_new_tokens=512,  # Shorter since we only need yes/no
            # do_sample=False  # Deterministic response
        )
        
        
# Extract response content
response = outputs[0]["generated_text"][-1]["content"].strip().lower()
response

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'let me work through this step-by-step:\n\n1. first, i\'ll formalize the premise: "there is at least one x for which if x writes a letter, then x is a lawyer and if x were a lawyer, then x is writing a letter."\n   - let l(x) = "x is a lawyer"\n   - let w(x) = "x writes a letter"\n   - the premise can be written as: ∀x (w(x) → l(x)) ∧ ∀x (l(x) → w(x))\n\n2. now, i\'ll formalize the query: "for all x, it is not true that x will write a letter if and only if x is a lawyer."\n   - let q(x) = "x will write a letter if and only if x is a lawyer"\n\n3. the query statement is logically equivalent to the negation of the implication: ¬(q(x) → l(x)) = ¬(¬q(x) ∨ l(x))\n   - this translates to: "it is not the case that if x will write a letter then x is a lawyer"\n\n4. now, i\'ll apply the rules of inference to derive the negation of the implication:\n   - ¬(q(x) → l(x)) ≡ q(x) ∧ ¬l(x)\n\n5. since we have established that the premise is true, we can infer the negation of the implication:\n   - the

In [47]:
import time
from tqdm.notebook import tqdm
from IPython.display import display, HTML

# Initialize results storage
results = []

# Run inference on samples with fixed progress tracking
for idx, row in tqdm(df_sample.iterrows(), total=len(df_sample), desc="ICL Inference"):
    # Construct user prompt from problem and query
    user_prompt = f"{row['problem']} {row['query']}"
    
    # Prepare messages for the model
    messages = [
        {"role": "system", "content": cot_system_prompt},
        {"role": "user", "content": cot_example_1_user},
        {"role": "assistant", "content": cot_example_1_assistant},
        {"role": "user", "content": cot_example_2_user},
        {"role": "assistant", "content": cot_example_2_assistant},
        {"role": "user", "content": user_prompt}
]
    
    # Get model response
    try:
        start_time = time.time()
        outputs = pipe(
            messages,
            max_new_tokens=512,  # Shorter since we only need yes/no
            # do_sample=False  # Deterministic response
        )
        inference_time = time.time() - start_time
        
        # Extract response content
        response = outputs[0]["generated_text"][-1]["content"].strip().lower()
        
        # Check if response contains yes or no
        if "yes" in response and "no" not in response:
            prediction = "yes"
        elif "no" in response:
            prediction = "no"
        else:
            prediction = "unclear"
            
        # Compare with ground truth
        correct = prediction == row["answer"]
        
        # Store results
        result = {
            "id": row["id"],
            "logic": row["logic"],
            "rule_category": row["rule_category"],
            "rule": row["rule"],
            "problem": row["problem"],
            "query": row["query"],
            "ground_truth": row["answer"],
            "prediction": prediction,
            "correct": correct,
            "inference_time": inference_time
        }
        results.append(result)
        
        # Add a small delay to avoid rate limiting
        # time.sleep(0.5)
        
    except Exception as e:
        print(f"Error processing row {idx}: {e}")

ICL Inference:   0%|          | 0/5000 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

In [48]:
results_df = pd.DataFrame(results)
results_df.head()

,id,logic,rule_category,rule,problem,query,ground_truth,prediction,correct,inference_time
0,1127,propositional,inference,disjunction elimination,unrelated,Consider the following premises: It is cloudy ...,no,no,True,1.587300
1,80,predicate,equivalent,Law of quantifier distribution,contradiction,Consider the following premises: There is at l...,no,no,True,2.264227
2,3261,propositional,inference,resolution,contradiction,Consider the following premises: Jennifer is r...,no,no,True,1.604278
3,1737,predicate,equivalent,existential biconditional laws,contradiction,Consider the following premises: There is at l...,no,no,True,2.590006
4,4527,predicate,equivalent,universal distributive laws,inference,"Consider the following premises: For all x, x ...",yes,no,False,2.738110


In [49]:
results_df.to_csv("results_cot.csv", index=False)

In [50]:
# Convert results to DataFrame for analysis
results_df = pd.DataFrame(results)

# Calculate overall accuracy
accuracy = results_df["correct"].mean() * 100
print(f"Overall Accuracy: {accuracy:.2f}%")

# Display number of samples by prediction
prediction_counts = results_df["prediction"].value_counts()
print("\nPrediction counts:")
for pred, count in prediction_counts.items():
    print(f"  {pred}: {count}")

# # Display accuracy by logic type
print("\nAccuracy by Logic Type:")
logic_acc = results_df.groupby("logic")["correct"].agg(["mean", "count"])
logic_acc["accuracy_pct"] = logic_acc["mean"] * 100
display(logic_acc[["count", "accuracy_pct"]])

# # Display accuracy by rule category
print("\nAccuracy by Rule Category:")
rule_acc = results_df.groupby("rule_category")["correct"].agg(["mean", "count"])
rule_acc["accuracy_pct"] = rule_acc["mean"] * 100
display(rule_acc.sort_values("count", ascending=False)[["count", "accuracy_pct"]])

# # Save results to CSV
results_file = "llama_predicate_logic_results.csv"
results_df.to_csv(results_file, index=False)
print(f"\nResults saved to {results_file}")

Overall Accuracy: 73.48%

Prediction counts:
  no: 4309
  yes: 682
  unclear: 9

Accuracy by Logic Type:


,count,accuracy_pct
logic,,
predicate,3511,73.483338
propositional,1489,73.472129



Accuracy by Rule Category:


,count,accuracy_pct
rule_category,,
inference,2591,70.513315
equivalent,1952,75.870902
fallacy,457,80.087527



Results saved to llama_predicate_logic_results.csv


In [70]:
results_df.value_counts("correct")

correct
False    612
True     388
Name: count, dtype: int64